# This code without Using Pipelines
Pipelines chains together multiple steps so that the output od each step is used as input to the next step. <br>
Pipelines makes it easy to apply the same preprocessing to train and test.

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.tree import DecisionTreeClassifier

In [2]:
df = pd.read_csv("E://Datasets/titanic.csv")
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


# Doing all the things without creating a pipeline.

In [3]:
df.drop(columns=['PassengerId','Name','Ticket','Cabin'],inplace=True)
df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S


In [4]:
## Train Test Split
x = df.drop(columns=['Survived'])
y = df['Survived']
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=42)

In [5]:
## Checking NULL values.
df.isnull().sum()

Survived      0
Pclass        0
Sex           0
Age         177
SibSp         0
Parch         0
Fare          0
Embarked      2
dtype: int64

In [6]:
## Applying Imputation
si_age = SimpleImputer()
si_embarked = SimpleImputer(strategy="most_frequent")

x_train_age = si_age.fit_transform(x_train[['Age']])
x_train_embarked = si_embarked.fit_transform(x_train[['Embarked']])

x_test_age = si_age.transform(x_test[['Age']])
x_test_embarked = si_embarked.transform(x_test[['Embarked']])

In [7]:
## One Hot Encoding

## handle_unknown='ignore': This means it will ignore every other new column.
ohe_sex = OneHotEncoder(sparse=False, handle_unknown='ignore')
ohe_embarked = OneHotEncoder(sparse=False,handle_unknown='ignore')

x_train_sex = ohe_sex.fit_transform(x_train[['Sex']])
x_train_embarked = ohe_embarked.fit_transform(x_train[['Embarked']])

x_test_sex = ohe_sex.transform(x_test[['Sex']])
x_test_embarked = ohe_embarked.transform(x_test[['Embarked']])

In [8]:
## Concatinating arrays as DataFrames.


## Dropping the columns which are converted into arrays and are preprocessed and ready to concant.
x_train_rem = x_train.drop(columns=['Sex','Age','Embarked'])
x_test_rem = x_test.drop(columns=['Sex','Age','Embarked'])

## Concatinating
x_train_transformed =  np.concatenate((x_train_rem,x_train_age,x_train_sex,x_train_embarked),axis=1)
x_test_transformed = np.concatenate((x_test_rem,x_test_age,x_test_sex,x_test_embarked),axis=1)

In [9]:
## Decision Tree Classifier.

## fitting 
clf = DecisionTreeClassifier()
clf.fit(x_train_transformed,y_train)

## prediction
y_pred = clf.predict(x_test_transformed)

## Accuracy.
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.7821229050279329

#### Now we will try to save the models, then load them and then we will predict.

In [10]:
## Saving models.

import pickle
import os
os.makedirs('models',exist_ok=True)
pickle.dump(clf,open('models/clf.pkl','wb'))
pickle.dump(si_embarked,open('models/si_embarked.pkl','wb'))
pickle.dump(si_age,open('models/si_age.pkl','wb'))
pickle.dump(ohe_embarked,open('models/ohe_embarked.pkl','wb'))
pickle.dump(ohe_sex,open('models/ohe_sex.pkl','wb'))

In [11]:
## loading model.
import pickle
import numpy as np

In [12]:
ohe_sex = pickle.load(open("models/ohe_sex.pkl","rb"))
ohe_embarked = pickle.load(open("models/ohe_embarked.pkl","rb"))
clf = pickle.load(open("models/clf.pkl","rb"))

In [13]:
## Test input
test_ip = np.array([[2,'male','31.0',0,0,10.5,"C"]],dtype=object)

In [14]:
test_ip_sex = ohe_sex.transform(test_ip[:,1].reshape(1,1))

C:\Users\sarth\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


In [15]:
test_ip_embarked = ohe_embarked.transform(test_ip[:,-1].reshape(1,1))

C:\Users\sarth\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


In [16]:
test_ip_age = test_ip[:,2].reshape(1,1)

In [17]:
# test_input[:,[0,3,4,5]] : dropping the sex , age and embarked columns.
test_ip_transformed = np.concatenate((test_ip[:,[0,3,4,5]],test_ip_age,test_ip_sex,test_ip_embarked),axis=1)

In [18]:
test_ip_transformed.shape

(1, 11)

In [19]:
clf.predict(test_ip_transformed)

array([0], dtype=int64)